In [1]:
def affine_gaps_align(seq1, seq2, opening, gap=-1, match=1, missmatch=-1):
    
    from itertools import product
    import numpy as np
    
    def cost(ch1, ch2):
        return match if ch1==ch2 else missmatch
    
    n=len(seq1)
    m=len(seq2)
    
    init=[n+1, m+1]
    
    Hor=np.zeros(init)#insertions
    Vert=np.zeros(init)#deletions
    Diag=np.zeros(init)
    Path=np.empty([3]+init)
    
    Hor[:,0]=[np.NINF]*(n+1)
    Hor[0,:]=[np.NINF]+[opening+i*gap for i in range(1, m+1)]
    Vert[0,:]=[np.NINF]*(m+1)
    Vert[:,0]=[np.NINF]+[opening+i*gap for i in range(1, n+1)]
    Diag[:,0]=[0]+[opening+i*gap for i in range(1, n+1)]
    Diag[0,:]=[0]+[opening+i*gap for i in range(1, m+1)]
    
    Path[1, :, 0]=[2]*(n+1)
    Path[1, 0, :]=[0]*(m+1)
    Path[0, :, 0]=[2]*(n+1)
    Path[0, 0, :]=[0]*(m+1)
    Path[2, :, 0]=[2]*(n+1)
    Path[2, 0, :]=[0]*(n+1)
    
    for i, j in product(range(1, n+1), range(1, m+1)):
        
        Hor[i,j]=max(Hor[i,j-1]+gap, Diag[i, j-1]+opening+gap)
        
        Path[0,i,j]=np.argmax([Hor[i,j-1]+gap, Diag[i, j-1]+opening+gap])
        
        Vert[i,j]=max(Diag[i-1, j]+opening+gap, Vert[i-1, j]+gap)
        
        Path[2,i,j]=np.argmax([np.NINF, Diag[i-1, j]+opening+gap, Vert[i-1, j]+gap])
        
        Diag[i,j]=max(Hor[i, j], Diag[i-1,j-1]+cost(seq1[i-1], seq2[j-1]), Vert[i, j])
        
        Path[1,i,j]=np.argmax([Hor[i, j], Diag[i-1,j-1]+cost(seq1[i-1], seq2[j-1]), Vert[i, j]])
        
    Alignment=[[],[]]
    
    while (i,j)!=(0,0):
        
        if Path[1, i, j]==1:
            
            Alignment[0].append(seq1[i-1])
            Alignment[1].append(seq2[j-1])
            i-=1
            j-=1
        elif Path[1,i,j]==0:
            k=1
            while Path[0,i,j-k+1]==0:
                if i==0 or j==0:
                    break
                k+=1
            Alignment[0]+=['-']*k
            Alignment[1]+=[seq2[l] for l in range(j-1,j-1-k,-1)]
            j-=k
            
        elif Path[1, i, j]==2: 
            k=1
            while Path[2,i-k+1,j]==0:
                if i==0 or j==0:
                    break
                k+=1
            Alignment[0]+=[seq1[l] for l in range(i-1,i-1-k,-1)]
            Alignment[1]+=['-']*k
            i-=k
        else: 
            raise ValueError('Wrong Path Matrix')
            
    Alignment=[x[::-1] for x in Alignment]
    Dist_mats=[Hor, Diag, Vert]
        
    return Dist_mats, Alignment, Path

In [23]:
D, A, P=affine_gaps_align('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC', 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC', 
                          opening=0, gap=-1, match=1, missmatch=-1)

In [24]:
print(D[1][D[1].shape[0]-1, D[1].shape[1]-1])
print(''.join(A[0]))
print(''.join(A[1]))

2.0
--T--CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
AATTGCCGCCGTCGT-TTTCA---G-CA-G-TTATGTCAGA-TC


In [25]:
from BioInf_Collection import Needleman_Wunsch
D, A=Needleman_Wunsch('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC', 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC', 
                          indel=-1, match=1, missmatch=-1)

In [26]:
print(D[D.shape[0]-1, D.shape[1]-1])
print(''.join(A[0]))
print(''.join(A[1]))

2.0
--T--CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
AATTGCCGCCGTCGTTT-TCAG----CA-GTT-ATGTCAGAT-C


Resulting alignment is not completely identical to Needlman-Wubsh's one, but it is an equivalent one. Difference arise due to the implementation details.

In [28]:
D, A, P=affine_gaps_align('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC', 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC', 
                          opening=-100, gap=-0.01, match=1, missmatch=-1)

In [29]:
print(D[1][D[1].shape[0]-1, D[1].shape[1]-1])
print(''.join(A[0]))
print(''.join(A[1]))

-20.0
TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC
AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC


Large penalty for opening the gap results in the absence of both insertions and deletions.

In [30]:
D, A, P=affine_gaps_align('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC', 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC', 
                          opening=0.5, gap=-0.3, match=1, missmatch=-1)

In [31]:
print(D[1][D[1].shape[0]-1, D[1].shape[1]-1])
print(''.join(A[0]))
print(''.join(A[1]))

27.69999999999999
--T--CC-CA-GT--TATGT-CAGGGGACACGAGC--ATG-CAGAGA-C
AATTGCCGC-CGTCGT-T-TTCA---G---C-AG-TTATGTC--AGATC


No missmatches in the above alignment, because subsequent gaps of length less than three have positive weight.